In [1]:
# =========================
# End-to-end ML pipeline for:
# A) Target inhibition (ChEMBL),
# B) Water solubility (TDC Solubility_AqSolDB),
# C) Human half-life (TDC Half_Life_Human),
# D) hERG inhibition (TDC hERG)
# =========================
from typing import List
import sys, subprocess

# ---------- Dependency management ----------
def ensure_installed(pkgs: List[str]):
    import importlib
    to_install = []
    for p in pkgs:
        mod_name = p.split("==")[0]
        try:
            importlib.import_module(mod_name.replace("-", "_"))
        except ImportError:
            to_install.append(p)
    if to_install:
        print("Installing missing packages:", to_install)
        subprocess.check_call([sys.executable, "-m", "pip", "install"] + to_install)

ensure_installed([
    "rdkit",       # CHANGES_MADE_01: rdkit-pypi -> rdkit
    "xgboost",
    "scikit-learn",
    "pandas",
    "numpy",
    "matplotlib",
    "seaborn",
    "tqdm",
    "requests",
    "PyTDC"
])

Installing missing packages: ['rdkit', 'scikit-learn', 'PyTDC']


In [4]:
!cp -r /content/drive/MyDrive/UAB/StanfordAI/data .
# !cp -r /content/drive/MyDrive/UAB/StanfordAI/figures .
# !cp -r /content/drive/MyDrive/UAB/StanfordAI/models .
# !cp -r /content/drive/MyDrive/UAB/StanfordAI/results .

In [1]:
# ---------- Imports ----------
import os, sys, time, json, math, random, subprocess, warnings, pickle, joblib
from typing import List, Tuple, Dict, Optional
warnings.filterwarnings("ignore")

from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import requests

from sklearn.preprocessing import RobustScaler
from sklearn.metrics import (
    roc_auc_score, average_precision_score, roc_curve, precision_recall_curve,
    confusion_matrix, f1_score, matthews_corrcoef, balanced_accuracy_score,
    brier_score_loss, r2_score, mean_absolute_error
)
from scipy.stats import spearmanr
from sklearn.utils import resample
from sklearn.inspection import permutation_importance

from xgboost import XGBClassifier, XGBRegressor

from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors, AllChem
from rdkit.Chem.Scaffolds import MurckoScaffold
from rdkit.Chem.MolStandardize import rdMolStandardize          # CHANGES_MADE_02 from rdkit.Chem import rdMolStandardize

# TDC
from tdc.single_pred import ADME

In [8]:
# ---------- Global config ----------
RANDOM_SEEDS = [13, 17, 23, 29, 31]  # 5 repeats
MODEL_DIR = "models"
RESULTS_DIR = "results"
FIG_DIR = "figures"
DATA_DIR = "data"
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(FIG_DIR, exist_ok=True)
os.makedirs(DATA_DIR, exist_ok=True)

TARGET = "any_target"

# ---------- Chemistry utilities ----------
def standardize_smiles(smiles: str) -> Optional[str]:
    if smiles is None or not isinstance(smiles, str) or len(smiles.strip()) == 0:
        return None
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None
        # Cleanup & largest fragment
        clean_mol = rdMolStandardize.Cleanup(mol)
        lfc = rdMolStandardize.LargestFragmentChooser()
        mol = lfc.choose(clean_mol)
        # Neutralize
        uncharger = rdMolStandardize.Uncharger()
        mol = uncharger.uncharge(mol)
        # Canonical SMILES
        can = Chem.MolToSmiles(mol, isomericSmiles=True, canonical=True)
        return can
    except Exception:
        return None

def compute_descriptors(smiles_list: List[str]) -> Tuple[pd.DataFrame, List[str]]:
    # Compute RDKit 2D descriptors
    desc_names = [d[0] for d in Descriptors.descList]
    records = []
    valid_idx = []
    for i, smi in enumerate(tqdm(smiles_list, desc="Computing RDKit 2D descriptors")):
        mol = Chem.MolFromSmiles(smi)
        if mol is None:
            records.append([np.nan]*len(desc_names))
            continue
        vals = []
        for (_, func) in Descriptors.descList:
            try:
                vals.append(func(mol))
            except Exception:
                vals.append(np.nan)
        records.append(vals)
        valid_idx.append(i)
    df = pd.DataFrame(records, columns=desc_names)
    # Remove columns that are all NaN or zero variance
    df = df.loc[:, ~df.isna().all()]
    nunique = df.nunique(dropna=True)
    df = df.loc[:, nunique > 1]
    # Replace inf with nan
    df = df.replace([np.inf, -np.inf], np.nan)
    # Impute with column median
    df = df.fillna(df.median(numeric_only=True))
    return df, list(df.columns)

def morgan_fp(smiles_list: List[str], radius=2, nBits=2048) -> np.ndarray:
    fps = np.zeros((len(smiles_list), nBits), dtype=np.uint8)
    for i, smi in enumerate(tqdm(smiles_list, desc=f"Computing MorganFP r{radius} n{nBits}")):
        mol = Chem.MolFromSmiles(smi)
        if mol is None:
            continue
        bv = rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, radius, nBits=nBits)
        arr = np.zeros((1, nBits), dtype=np.uint8)
        Chem.DataStructs.ConvertToNumpyArray(bv, arr[0])
        fps[i, :] = arr
    return fps

def murcko_scaffold(smiles: str) -> str:
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return ""
        scaf = MurckoScaffold.MurckoScaffoldSmiles(mol=mol, includeChirality=False)
        if scaf is None:
            return ""
        return scaf
    except Exception:
        return ""

def scaffold_split_indices(smiles: List[str], train_frac=0.72, val_frac=0.08, test_frac=0.20, seed=42):
    assert abs(train_frac + val_frac + test_frac - 1.0) < 1e-6
    rng = np.random.RandomState(seed)
    scaffolds = {}
    for idx, smi in enumerate(smiles):
        scaf = murcko_scaffold(smi)
        scaffolds.setdefault(scaf, []).append(idx)
    # Shuffle scaffolds deterministically
    scaf_items = list(scaffolds.items())
    rng.shuffle(scaf_items)
    # Sort by scaffold size (large first) to fill splits
    scaf_items.sort(key=lambda x: len(x[1]), reverse=True)

    n = len(smiles)
    train_idx, val_idx, test_idx = [], [], []
    train_target = int(train_frac * n)
    val_target = int(val_frac * n)
    # Greedy fill
    for scaf, idxs in scaf_items:
        if len(train_idx) + len(idxs) <= train_target:
            train_idx.extend(idxs)
        elif len(val_idx) + len(idxs) <= val_target:
            val_idx.extend(idxs)
        else:
            test_idx.extend(idxs)
    # In case of rounding leftovers, move extras to test
    leftover = set(range(n)) - set(train_idx) - set(val_idx) - set(test_idx)
    test_idx.extend(list(leftover))
    return np.array(train_idx), np.array(val_idx), np.array(test_idx)

# ---------- Feature builder ----------
def build_features(smiles: List[str], radius=2, nBits=2048):
    df_desc, desc_cols = compute_descriptors(smiles)
    scaler = RobustScaler()
    X_desc = scaler.fit_transform(df_desc.values.astype(float))
    X_fp = morgan_fp(smiles, radius=radius, nBits=nBits).astype(np.float32)
    X = np.hstack([X_desc, X_fp])
    feature_meta = {
        "desc_cols": desc_cols,
        "desc_scaler_center_": scaler.center_.tolist(),
        "desc_scaler_scale_": scaler.scale_.tolist(),
        "fp_radius": radius,
        "fp_nBits": nBits,
        "n_desc": len(desc_cols),
        "n_fp": nBits,
        "n_total": X.shape[1]
    }
    return X, df_desc, scaler, feature_meta

# ---------- Evaluation ----------
def classification_metrics(y_true, y_proba, threshold=0.5):
    y_pred = (y_proba >= threshold).astype(int)
    metrics = {}
    try:
        metrics["auroc"] = roc_auc_score(y_true, y_proba)
    except Exception:
        metrics["auroc"] = np.nan
    try:
        metrics["auprc"] = average_precision_score(y_true, y_proba)
    except Exception:
        metrics["auprc"] = np.nan
    metrics["f1"] = f1_score(y_true, y_pred, zero_division=0)
    metrics["mcc"] = matthews_corrcoef(y_true, y_pred)
    metrics["balanced_acc"] = balanced_accuracy_score(y_true, y_pred)
    try:
        metrics["brier"] = brier_score_loss(y_true, y_proba)
    except Exception:
        metrics["brier"] = np.nan
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()
    metrics.update({"tn": tn, "fp": fp, "fn": fn, "tp": tp, "threshold": threshold})
    return metrics

def regression_metrics(y_true, y_pred):
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    try:
        rho, _ = spearmanr(y_true, y_pred)
    except Exception:
        rho = np.nan
    return {"rmse": rmse, "mae": mae, "r2": r2, "spearman": rho}

# ---------- Plotting ----------
def set_pub_style():
    plt.style.use("seaborn-v0_8-whitegrid")           # CHANGES_MADE_04: seaborn-whitegrid
    sns.set_context("talk")
    plt.rcParams["figure.dpi"] = 150
    plt.rcParams["savefig.dpi"] = 300
    plt.rcParams["font.size"] = 12
    plt.rcParams["axes.labelsize"] = 12
    plt.rcParams["legend.fontsize"] = 10
    plt.rcParams["axes.titlesize"] = 14

def plot_roc_pr_curves(y_true_list, y_proba_list, title_prefix, out_prefix):
    set_pub_style()
    # ROC
    plt.figure(figsize=(6,5))
    for i, (yt, yp) in enumerate(zip(y_true_list, y_proba_list)):
        fpr, tpr, _ = roc_curve(yt, yp)
        auc = roc_auc_score(yt, yp)
        plt.plot(fpr, tpr, lw=1.5, label=f"Split {i+1} (AUROC={auc:.3f})")
    plt.plot([0,1], [0,1], 'k--', lw=1)
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(f"{title_prefix}: ROC Curves")
    plt.legend(frameon=True)
    plt.tight_layout()
    plt.savefig(f"{FIG_DIR}/{out_prefix}_roc.png")
    plt.close()

    # PR
    plt.figure(figsize=(6,5))
    for i, (yt, yp) in enumerate(zip(y_true_list, y_proba_list)):
        prec, rec, _ = precision_recall_curve(yt, yp)
        aupr = average_precision_score(yt, yp)
        plt.plot(rec, prec, lw=1.5, label=f"Split {i+1} (AUPRC={aupr:.3f})")
    base = np.mean(np.concatenate(y_true_list))
    plt.hlines(base, 0, 1, colors='k', linestyles='--', label=f"Baseline={base:.3f}")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title(f"{title_prefix}: Precision-Recall Curves")
    plt.legend(frameon=True)
    plt.tight_layout()
    plt.savefig(f"{FIG_DIR}/{out_prefix}_pr.png")
    plt.close()

def plot_conf_matrix(y_true, y_pred, title, out_path):
    set_pub_style()
    cm = confusion_matrix(y_true, y_pred, labels=[0,1])
    plt.figure(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False,
                xticklabels=["Pred 0","Pred 1"], yticklabels=["True 0","True 1"])
    plt.title(title)
    plt.ylabel("True label")
    plt.xlabel("Predicted label")
    plt.tight_layout()
    plt.savefig(out_path)
    plt.close()

def plot_parity(y_true_list, y_pred_list, title, out_path, xlabel="True", ylabel="Predicted"):
    set_pub_style()
    plt.figure(figsize=(5.5,5))
    for i, (yt, yp) in enumerate(zip(y_true_list, y_pred_list)):
        plt.scatter(yt, yp, s=12, alpha=0.5, label=f"Split {i+1}")
    mn = min(np.min(np.concatenate(y_true_list)), np.min(np.concatenate(y_pred_list)))
    mx = max(np.max(np.concatenate(y_true_list)), np.max(np.concatenate(y_pred_list)))
    plt.plot([mn, mx], [mn, mx], 'k--', lw=1)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend(frameon=True, markerscale=1, fontsize=9)
    plt.tight_layout()
    plt.savefig(out_path)
    plt.close()

def plot_residuals(y_true_list, y_pred_list, title, out_path):
    set_pub_style()
    plt.figure(figsize=(6,4))
    res_all = []
    for yt, yp in zip(y_true_list, y_pred_list):
        res_all.append(yp - yt)
    res = np.concatenate(res_all)
    sns.histplot(res, bins=40, kde=True, color="slateblue")
    plt.axvline(0, color="k", linestyle="--", lw=1)
    plt.title(title)
    plt.xlabel("Residual (Pred - True)")
    plt.ylabel("Count")
    plt.tight_layout()
    plt.savefig(out_path)
    plt.close()

def plot_feature_importance(model, X_cols, X, y, title, out_path, n_top=20, random_state=42):
    set_pub_style()
    # Permutation importance on descriptor block only (interpretable)
    r = permutation_importance(model, X, y, n_repeats=10, random_state=random_state, scoring=None)
    importances = pd.Series(r.importances_mean, index=X_cols)
    imp = importances.sort_values(ascending=False).head(n_top)
    plt.figure(figsize=(6, 6))
    sns.barplot(x=imp.values, y=imp.index, color="teal")
    plt.xlabel("Permutation importance (mean decrease in score)")
    plt.ylabel("Feature")
    plt.title(title)
    plt.tight_layout()
    plt.savefig(out_path)
    plt.close()

# ---------- Dataset loaders ----------
def load_tdc_solubility():
    print("Loading TDC Solubility_AqSolDB...")
    data = ADME(name="Solubility_AqSolDB").get_data()
    # columns: Drug (SMILES), Y (logS)
    df = pd.DataFrame(data)
    df = df.rename(columns={"Drug": "smiles", "Y": "y"})
    return df

def load_tdc_half_life():
    print("Loading TDC Half_Life_Obach...")
    data = ADME(name="Half_Life_Obach").get_data()      # CHANGES_MADE_06 Half_Life_Obach from Half_Life_Human
    df = pd.DataFrame(data)
    df = df.rename(columns={"Drug": "smiles", "Y": "y"})
    return df

def load_tdc_herg():
    print("Loading TDC hERG...")

    from tdc.single_pred import Tox
    data = Tox(name = 'hERG_Karim').get_data()

    # data = ADME(name="hERG_Karim").get_data()                 # CHANGES_MADE_08: hERG_Karim from hERG; Tox to ADME
    df = pd.DataFrame(data)
    df = df.rename(columns={"Drug": "smiles", "Y": "y"})
    return df

# ---------- ChEMBL ----------
CHEMBL_API = "https://www.ebi.ac.uk/chembl/api/data"

def chembl_get_target_id(query_target):
    query_target = query_target.replace(" ", "%20")
    url = f"{CHEMBL_API}/target/search.json?q={query_target}"
    r = requests.get(url, timeout=30)
    r.raise_for_status()
    js = r.json()
    for rec in js.get("targets", []):
        if rec.get("target_type") == "SINGLE PROTEIN" and rec.get("organism") == "Homo sapiens":
            return rec.get("target_chembl_id")
    # fallback: just choose the first
    if js.get("targets"):
        return js["targets"][0]["target_chembl_id"]
    raise RuntimeError(f"Target {query_target} not found in ChEMBL search.")

def chembl_fetch_activities_for_target(target_id, page_size=1000, max_pages=50):
    # get activities with pchembl_value present if possible
    activities = []
    for page in range(max_pages):
        offset = page * page_size
        url = f"{CHEMBL_API}/activity.json?target_chembl_id={target_id}&limit={page_size}&offset={offset}"
        r = requests.get(url, timeout=60)
        r.raise_for_status()
        js = r.json()
        items = js.get("activities", [])
        if not items:
            break
        activities.extend(items)
    return activities

def chembl_fetch_smiles_for_molecules(mol_ids: List[str], sleep=0.05):
    smiles_map = {}
    for mid in tqdm(mol_ids, desc="Fetching SMILES from ChEMBL"):
        url = f"{CHEMBL_API}/molecule/{mid}.json"
        r = requests.get(url, timeout=30)
        if r.status_code != 200:
            continue
        js = r.json()
        smi = None
        try:
            smi = js.get("molecule_structures", {}).get("canonical_smiles", None)
        except Exception:
            smi = None
        smiles_map[mid] = smi
        time.sleep(sleep)
    return smiles_map

def load_chembl_data(cache_path):
    if os.path.exists(cache_path):
        print(f"Loading cached {TARGET} data from {cache_path}")
        return pd.read_csv(cache_path)
    print(f"Querying ChEMBL for {TARGET} activities...")
    tid = chembl_get_target_id(TARGET)
    acts = chembl_fetch_activities_for_target(tid)
    rows = []
    mol_ids = set()
    for a in acts:
        pchem = a.get("pchembl_value")
        rel = a.get("standard_relation", None)
        stype = a.get("standard_type", None)
        svalue = a.get("standard_value", None)
        sunits = a.get("standard_units", None)
        mid = a.get("molecule_chembl_id", None)
        if mid is None:
            continue
        mol_ids.add(mid)
        rows.append({
            "molecule_chembl_id": mid,
            "pchembl_value": float(pchem) if pchem is not None else np.nan,
            "standard_relation": rel,
            "standard_type": stype,
            "standard_value": float(svalue) if svalue is not None else np.nan,
            "standard_units": sunits
        })
    df = pd.DataFrame(rows)
    # Fetch SMILES
    smiles_map = chembl_fetch_smiles_for_molecules(sorted(mol_ids))
    df["smiles"] = df["molecule_chembl_id"].map(smiles_map)
    # Filter rows with smiles
    df = df[~df["smiles"].isna()].copy()
    # Prefer pchembl_value; if missing, try to compute pChEMBL-like value for IC50/Ki in nM
    def to_pchembl(row):
        if not np.isnan(row["pchembl_value"]):
            return row["pchembl_value"]
        stype = str(row["standard_type"]).upper() if row["standard_type"] is not None else ""
        sval = row["standard_value"]
        sunits = str(row["standard_units"]).lower() if row["standard_units"] is not None else ""
        rel = row["standard_relation"]
        # Only accept numeric equals or less-than
        if rel not in ["=", "<", "<="]:
            return np.nan
        if np.isnan(sval):
            return np.nan
        # Convert to nM when possible
        factor = None
        if sunits in ["nm", "nanomolar", "nм"]:  # handle common
            factor = 1.0
        elif sunits in ["um", "µm", "micromolar"]:
            factor = 1e3
        elif sunits in ["mm", "millimolar"]:
            factor = 1e6
        else:
            return np.nan
        if stype in ["IC50", "KI", "EC50", "XC50"]:
            # pChEMBL ~ -log10(M)
            nM = sval * factor
            if nM <= 0:
                return np.nan
            M = nM * 1e-9
            return -math.log10(M)
        return np.nan
    df["pchembl"] = df.apply(to_pchembl, axis=1)
    df = df[~df["pchembl"].isna()].copy()
    df.to_csv(cache_path, index=False)
    return df

# ---------- Task pipelines ----------
def prepare_dataframe_generic(df_in: pd.DataFrame, task: str) -> pd.DataFrame:
    # Standardize SMILES, deduplicate
    df = df_in.copy()
    df["smiles_std"] = [standardize_smiles(s) for s in tqdm(df["smiles"].tolist(), desc=f"Standardizing SMILES for {task}")]
    df = df[~df["smiles_std"].isna()].copy()
    df = df.drop_duplicates(subset=["smiles_std"])
    df = df.reset_index(drop=True)
    return df

def prepare_bioactivity_classification(pchembl_threshold_active=6.0, min_positives=50):
    raw = load_chembl_data(f"{DATA_DIR}/chembl_cache_{TARGET}.csv")
    df = raw[["smiles", "pchembl"]].copy()
    df = df.dropna()
    # Standardize and aggregate
    df["smiles_std"] = [standardize_smiles(s) for s in tqdm(df["smiles"].tolist(), desc=f"Standardizing SMILES for {TARGET}")]
    df = df[~df["smiles_std"].isna()].copy()
    agg = df.groupby("smiles_std")["pchembl"].median().reset_index()
    threshold = pchembl_threshold_active
    # Adjust threshold if too few positives
    n_pos = (agg["pchembl"] >= threshold).sum()
    if n_pos < min_positives:
        threshold = 5.5
        print(f"{TARGET}: Not enough positives at pChEMBL≥{pchembl_threshold_active}. Relaxing to {threshold}.")
    agg["y"] = (agg["pchembl"] >= threshold).astype(int)
    agg = agg.rename(columns={"smiles_std": "smiles"})
    # persist processed dataset
    outp = f"{DATA_DIR}/{TARGET}_processed.csv"
    agg.to_csv(outp, index=False)
    print(f"{TARGET} processed saved to {outp} | N={len(agg)}; Positives={agg['y'].sum()}")
    return agg, threshold

def prepare_solubility_regression():
    df_raw = load_tdc_solubility()
    df = prepare_dataframe_generic(df_raw, "Solubility")
    df = df.drop(columns=["smiles"]) # CHANGES_MADE_05: Remove original smiles column
    df = df.rename(columns={"smiles_std": "smiles", "y": "logS"})
    outp = f"{DATA_DIR}/solubility_processed.csv"
    df[["smiles", "logS"]].to_csv(outp, index=False)
    print(f"Solubility processed saved to {outp} | N={len(df)}")
    return df[["smiles", "logS"]]

def prepare_half_life_regression():
    df_raw = load_tdc_half_life()
    df = prepare_dataframe_generic(df_raw, "HalfLife")
    df = df.drop(columns=["smiles"]) # CHANGES_MADE_07: Remove original smiles column
    # log10-transform (add small epsilon to avoid log of zero)
    eps = 1e-6
    df["log10_half_life"] = np.log10(df["y"].astype(float) + eps)
    df = df.rename(columns={"smiles_std": "smiles"})
    outp = f"{DATA_DIR}/half_life_processed.csv"
    df[["smiles", "log10_half_life"]].to_csv(outp, index=False)
    print(f"Half-Life processed saved to {outp} | N={len(df)}")
    return df[["smiles", "log10_half_life"]]

def prepare_herg_classification():
    df_raw = load_tdc_herg()
    df = prepare_dataframe_generic(df_raw, "hERG")
    df = df.drop(columns=["smiles"]) # CHANGES_MADE_09: Remove original smiles column
    df = df.rename(columns={"smiles_std": "smiles", "y": "label"})
    outp = f"{DATA_DIR}/herg_processed.csv"
    df[["smiles", "label"]].to_csv(outp, index=False)
    print(f"hERG processed saved to {outp} | N={len(df)}; Positives={df['label'].sum()}")
    return df[["smiles", "label"]]

# ---------- Training and benchmarking ----------
def tune_xgb_classifier(X_train, y_train, X_val, y_val, n_iter=20, seed=42, scale_pos_weight=1.0):
    rng = np.random.RandomState(seed)
    best = None
    best_score = -np.inf
    for i in range(n_iter):
        params = {
            "n_estimators": int(rng.randint(300, 1200)),
            "max_depth": int(rng.randint(3, 9)),
            "learning_rate": 10 ** rng.uniform(-2.0, -0.3),  # 0.01 - 0.5
            "subsample": rng.uniform(0.6, 1.0),
            "colsample_bytree": rng.uniform(0.5, 1.0),
            "min_child_weight": rng.uniform(1.0, 8.0),
            "reg_lambda": 10 ** rng.uniform(-3, 1),  # 0.001 - 10
            "reg_alpha": 10 ** rng.uniform(-4, 0),  # 0.0001 - 1
            "gamma": rng.uniform(0.0, 5.0),
            "scale_pos_weight": scale_pos_weight,
            "random_state": seed,
            "eval_metric": "aucpr",
            "n_jobs": max(1, os.cpu_count() // 2)
        }
        model = XGBClassifier(**params, tree_method="hist", use_label_encoder=False)
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)       # CHANGES_MADE_03: removed early_stopping_rounds=50
        proba = model.predict_proba(X_val)[:, 1]
        score = average_precision_score(y_val, proba)
        if score > best_score:
            best_score = score
            best = model
    return best

def tune_xgb_regressor(X_train, y_train, X_val, y_val, n_iter=20, seed=42):
    rng = np.random.RandomState(seed)
    best = None
    best_score = np.inf  # RMSE
    for i in range(n_iter):
        params = {
            "n_estimators": int(rng.randint(300, 1200)),
            "max_depth": int(rng.randint(3, 9)),
            "learning_rate": 10 ** rng.uniform(-2.0, -0.3),  # 0.01 - 0.5
            "subsample": rng.uniform(0.6, 1.0),
            "colsample_bytree": rng.uniform(0.5, 1.0),
            "min_child_weight": rng.uniform(1.0, 8.0),
            "reg_lambda": 10 ** rng.uniform(-3, 1),
            "reg_alpha": 10 ** rng.uniform(-4, 0),
            "gamma": rng.uniform(0.0, 5.0),
            "random_state": seed,
            "objective": "reg:squarederror",
            "n_jobs": max(1, os.cpu_count() // 2)
        }
        model = XGBRegressor(**params, tree_method="hist")
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)              # CHANGES_MADE_03: removed early_stopping_rounds=50
        pred = model.predict(X_val)
        rmse = np.sqrt(np.mean((pred - y_val) ** 2))
        if rmse < best_score:
            best_score = rmse
            best = model
    return best

def find_best_threshold(y_true, y_proba):
    # Maximize F1
    pr, rc, thr = precision_recall_curve(y_true, y_proba)
    # Convert to thresholds on proba grid
    # Select threshold that maximizes F1 (on PR curve points, threshold len = len(rc)-1)
    best_f1, best_t = 0.0, 0.5
    for t in np.linspace(0.05, 0.95, 19):
        yp = (y_proba >= t).astype(int)
        f1 = f1_score(y_true, yp, zero_division=0)
        if f1 > best_f1:
            best_f1, best_t = f1, t
    return best_t

def run_classification_task(name: str, df: pd.DataFrame, label_col: str):
    # Build features
    smiles = df["smiles"].tolist()
    X, df_desc, scaler, fmeta = build_features(smiles)
    X[X > 1e5] = 1e5

    # for testing
    joblib.dump(scaler, f"models/{name}_desc_scaler.pkl")
    with open(f"models/{name}_feature_meta.json", "w") as f:
        json.dump(fmeta, f, indent=4)

    y = df[label_col].values.astype(int)
    # Keep descriptor info for permutation importance
    n_desc = fmeta["n_desc"]
    desc_cols = df_desc.columns.tolist()

    # Repeated scaffold splits
    metrics_list = []
    roc_y_true, roc_y_proba = [], []
    cm_images = []
    featimp_done = False

    for i, seed in enumerate(RANDOM_SEEDS):
        print(f"[{name}] Split {i+1}/{len(RANDOM_SEEDS)} with seed {seed}")
        tr_idx, va_idx, te_idx = scaffold_split_indices(smiles, train_frac=0.72, val_frac=0.08, test_frac=0.20, seed=seed)
        X_tr, y_tr = X[tr_idx], y[tr_idx]
        X_va, y_va = X[va_idx], y[va_idx]
        X_te, y_te = X[te_idx], y[te_idx]

        # Class imbalance
        n_pos = y_tr.sum()
        n_neg = len(y_tr) - n_pos
        spw = float(n_neg) / max(1.0, float(n_pos))

        # Tune and train
        model = tune_xgb_classifier(X_tr, y_tr, X_va, y_va, n_iter=20, seed=seed, scale_pos_weight=spw)
        with open(f'models/{name}_{seed}.pkl', 'wb') as f: pickle.dump(model, f)

        # Validate threshold
        val_proba = model.predict_proba(X_va)[:, 1]
        thr = find_best_threshold(y_va, val_proba)
        # Test predictions
        te_proba = model.predict_proba(X_te)[:, 1]
        te_pred = (te_proba >= thr).astype(int)

        # Metrics
        met = classification_metrics(y_te, te_proba, threshold=thr)
        met["split_seed"] = seed
        metrics_list.append(met)

        roc_y_true.append(y_te)
        roc_y_proba.append(te_proba)

        # Confusion matrix per split
        cm_path = f"{FIG_DIR}/{name}_cm_split{i+1}.png"
        plot_conf_matrix(y_te, te_pred, f"{name} Confusion Matrix (split {i+1})", cm_path)
        cm_images.append(cm_path)

        # Permutation importance (only once on the first split for descriptor block)
        if not featimp_done:
            # Build a model on descriptors only to compute interpretable importances
            model_desc = XGBClassifier(
                n_estimators=model.get_params()["n_estimators"],
                max_depth=model.get_params()["max_depth"],
                learning_rate=model.get_params()["learning_rate"],
                subsample=model.get_params()["subsample"],
                colsample_bytree=model.get_params()["colsample_bytree"],
                min_child_weight=model.get_params()["min_child_weight"],
                reg_lambda=model.get_params()["reg_lambda"],
                reg_alpha=model.get_params()["reg_alpha"],
                gamma=model.get_params()["gamma"],
                scale_pos_weight=spw,
                random_state=seed,
                eval_metric="aucpr",
                n_jobs=max(1, os.cpu_count() // 2),
                tree_method="hist",
                use_label_encoder=False
            )
            # Train on descriptor-only slice
            X_tr_desc = X_tr[:, :n_desc]
            X_te_desc = X_te[:, :n_desc]
            model_desc.fit(X_tr_desc, y_tr)
            plot_feature_importance(
                model_desc, desc_cols, X_te_desc, y_te,
                title=f"{name}: Descriptor Permutation Importance (test split 1)",
                out_path=f"{FIG_DIR}/{name}_featimp_desc_split1.png",
                n_top=20,
                random_state=seed
            )
            featimp_done = True

    # Aggregate metrics
    metrics_df = pd.DataFrame(metrics_list)
    out_csv = f"{RESULTS_DIR}/{name}_classification_metrics.csv"
    metrics_df.to_csv(out_csv, index=False)
    print(f"[{name}] Metrics saved to {out_csv}")
    # ROC/PR curves
    plot_roc_pr_curves(roc_y_true, roc_y_proba, f"{name}", f"{name}")
    print(f"[{name}] ROC and PR curves saved to {FIG_DIR}/{name}_*.png")

def run_regression_task(name: str, df: pd.DataFrame, target_col: str, xlabel: str):
    smiles = df["smiles"].tolist()
    X, df_desc, scaler, fmeta = build_features(smiles)
    X[X > 1e5] = 1e5

    # for testing
    joblib.dump(scaler, f"models/{name}_desc_scaler.pkl")
    with open(f"models/{name}_feature_meta.json", "w") as f:
        json.dump(fmeta, f, indent=4)

    y = df[target_col].values.astype(float)
    n_desc = fmeta["n_desc"]
    desc_cols = df_desc.columns.tolist()

    metrics_list = []
    ytrue_list, ypred_list = [], []
    featimp_done = False

    for i, seed in enumerate(RANDOM_SEEDS):
        print(f"[{name}] Split {i+1}/{len(RANDOM_SEEDS)} with seed {seed}")
        tr_idx, va_idx, te_idx = scaffold_split_indices(smiles, train_frac=0.72, val_frac=0.08, test_frac=0.20, seed=seed)
        X_tr, y_tr = X[tr_idx], y[tr_idx]
        X_va, y_va = X[va_idx], y[va_idx]
        X_te, y_te = X[te_idx], y[te_idx]

        model = tune_xgb_regressor(X_tr, y_tr, X_va, y_va, n_iter=20, seed=seed)
        with open(f'models/{name}_{seed}.pkl', 'wb') as f: pickle.dump(model, f)

        y_pred = model.predict(X_te)

        met = regression_metrics(y_te, y_pred)
        met["split_seed"] = seed
        metrics_list.append(met)

        ytrue_list.append(y_te)
        ypred_list.append(y_pred)

        if not featimp_done:
            # Permutation importance on descriptors
            model_desc = XGBRegressor(
                n_estimators=model.get_params()["n_estimators"],
                max_depth=model.get_params()["max_depth"],
                learning_rate=model.get_params()["learning_rate"],
                subsample=model.get_params()["subsample"],
                colsample_bytree=model.get_params()["colsample_bytree"],
                min_child_weight=model.get_params()["min_child_weight"],
                reg_lambda=model.get_params()["reg_lambda"],
                reg_alpha=model.get_params()["reg_alpha"],
                gamma=model.get_params()["gamma"],
                random_state=seed,
                objective="reg:squarederror",
                n_jobs=max(1, os.cpu_count() // 2),
                tree_method="hist"
            )
            X_tr_desc = X_tr[:, :n_desc]
            X_te_desc = X_te[:, :n_desc]
            model_desc.fit(X_tr_desc, y_tr)
            plot_feature_importance(
                model_desc, desc_cols, X_te_desc, y_te,
                title=f"{name}: Descriptor Permutation Importance (test split 1)",
                out_path=f"{FIG_DIR}/{name}_featimp_desc_split1.png",
                n_top=20,
                random_state=seed
            )
            featimp_done = True

    # Aggregate and save
    metrics_df = pd.DataFrame(metrics_list)
    out_csv = f"{RESULTS_DIR}/{name}_regression_metrics.csv"
    metrics_df.to_csv(out_csv, index=False)
    print(f"[{name}] Metrics saved to {out_csv}")

    # Plots
    plot_parity(ytrue_list, ypred_list, title=f"{name}: Parity Plot", out_path=f"{FIG_DIR}/{name}_parity.png", xlabel=xlabel, ylabel="Predicted")
    plot_residuals(ytrue_list, ypred_list, title=f"{name}: Residuals", out_path=f"{FIG_DIR}/{name}_residuals.png")
    print(f"[{name}] Parity and residual plots saved to {FIG_DIR}/{name}_*.png")

def build_features_for_test(smiles: List[str], fmeta, scaler):
    df_desc_new, desc_cols_new = compute_descriptors(smiles)
    df_desc_new = df_desc_new.reindex(columns=fmeta["desc_cols"], fill_value=0.0)
    X_desc_new = scaler.transform(df_desc_new.values.astype(float))
    X_fp_new = morgan_fp(smiles, radius=fmeta["fp_radius"], nBits=fmeta["fp_nBits"]).astype(np.float32)
    X_new = np.hstack([X_desc_new, X_fp_new])
    return X_new, df_desc_new

def test_inference(smiles, task_name):
    # load scaler + metadata
    scaler = joblib.load(f"models/{task_name}_desc_scaler.pkl")
    with open(f"models/{task_name}_feature_meta.json", "r") as f:
        fmeta = json.load(f)

    X_test, df_desc_test = build_features_for_test(smiles, fmeta, scaler)

    print()
    for s in RANDOM_SEEDS:
        with open(f'models/{task_name}_{s}.pkl', 'rb') as f:
            loaded_model = pickle.load(f)
            print(loaded_model.predict(X_test))

# ---------- Main orchestration ----------

In [19]:
threshold_dict = {
    "SGLT2": 7.8,
    "CGAS": 6.0,
    "SEH": 7.8,
    "HDAC": 6.5,
    "DYRK1A": 7.2
}

In [23]:
# RANDOM_SEEDS = [13]
RANDOM_SEEDS = [13, 17, 23, 29, 31]  # 5 repeats

for t in ["SGLT2", "CGAS", "SEH", "HDAC", "DYRK1A"]:
    TARGET = t
    print(f"https://www.ebi.ac.uk/chembl/explore/target/{chembl_get_target_id(TARGET)}")

    training_name = f"{TARGET}_inhibition"
    bioactivity_df, bioactivity_thr = prepare_bioactivity_classification(pchembl_threshold_active=threshold_dict[TARGET])
    print(bioactivity_df["y"].value_counts(), bioactivity_thr)

    run_classification_task(training_name, bioactivity_df[["smiles","y"]].rename(columns={"y": "label"}), label_col="label")

    test_smiles = ['CCO', 'CCN', 'CC(=O)O']
    test_inference(test_smiles, training_name)

https://www.ebi.ac.uk/chembl/explore/target/CHEMBL3884
Loading cached SGLT2 data from data/chembl_cache_SGLT2.csv


Standardizing SMILES for SGLT2: 100%|██████████| 1762/1762 [00:04<00:00, 370.27it/s]


SGLT2 processed saved to data/SGLT2_processed.csv | N=1437; Positives=638
y
0    799
1    638
Name: count, dtype: int64 7.8


Computing MorganFP r2 n2048: 100%|██████████| 1437/1437 [00:00<00:00, 1955.45it/s]


[SGLT2_inhibition] Split 1/5 with seed 13
[SGLT2_inhibition] Split 2/5 with seed 17
[SGLT2_inhibition] Split 3/5 with seed 23
[SGLT2_inhibition] Split 4/5 with seed 29
[SGLT2_inhibition] Split 5/5 with seed 31
[SGLT2_inhibition] Metrics saved to results/SGLT2_inhibition_classification_metrics.csv
[SGLT2_inhibition] ROC and PR curves saved to figures/SGLT2_inhibition_*.png


...

DYRK1A processed saved to data/DYRK1A_processed.csv | N=2764; Positives=1211
y
0    1553
1    1211
Name: count, dtype: int64 7.2


Computing MorganFP r2 n2048: 100%|██████████| 2764/2764 [00:01<00:00, 2195.78it/s]


[DYRK1A_inhibition] Split 1/5 with seed 13
[DYRK1A_inhibition] Split 2/5 with seed 17
[DYRK1A_inhibition] Split 3/5 with seed 23
[DYRK1A_inhibition] Split 4/5 with seed 29
[DYRK1A_inhibition] Split 5/5 with seed 31
[DYRK1A_inhibition] Metrics saved to results/DYRK1A_inhibition_classification_metrics.csv
[DYRK1A_inhibition] ROC and PR curves saved to figures/DYRK1A_inhibition_*.png


Computing MorganFP r2 n2048: 100%|██████████| 3/3 [00:00<00:00, 2835.27it/s]


[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]


In [ ]:
# B) Water solubility (regression)
sol_df = prepare_solubility_regression()
run_regression_task("Solubility_AqSolDB", sol_df, target_col="logS", xlabel="logS (mol/L)")

Found local copy...
Loading...
Done!


Loading TDC Solubility_AqSolDB...


Standardizing SMILES for Solubility: 100%|██████████| 9982/9982 [00:24<00:00, 408.92it/s]


Solubility processed saved to data/solubility_processed.csv | N=9458


Computing MorganFP r2 n2048: 100%|██████████| 9458/9458 [00:04<00:00, 2318.02it/s]


[Solubility_AqSolDB] Split 1/5 with seed 13
[Solubility_AqSolDB] Split 2/5 with seed 17
[Solubility_AqSolDB] Split 3/5 with seed 23
[Solubility_AqSolDB] Split 4/5 with seed 29
[Solubility_AqSolDB] Split 5/5 with seed 31
[Solubility_AqSolDB] Metrics saved to results/Solubility_AqSolDB_regression_metrics.csv
[Solubility_AqSolDB] Parity and residual plots saved to figures/Solubility_AqSolDB_*.png


In [ ]:
test_inference(smiles, "Solubility_AqSolDB")

Computing MorganFP r2 n2048: 100%|██████████| 3/3 [00:00<00:00, 2574.25it/s]



[0.9059403  1.15754    0.49541146]
[0.9341944  1.4734123  0.24018402]
[0.74191403 0.84268844 0.08887617]
[0.7371227  0.82696044 0.42432117]
[0.82666254 1.2370104  0.30237755]


In [ ]:
# C) Human half-life (regression; log10 transformed)
hl_df = prepare_half_life_regression()
run_regression_task("HalfLife_Human", hl_df, target_col="log10_half_life", xlabel="log10(t1/2)")

Found local copy...
Loading...
Done!


Loading TDC Half_Life_Obach...


Standardizing SMILES for HalfLife: 100%|██████████| 667/667 [00:01<00:00, 387.19it/s]


Half-Life processed saved to data/half_life_processed.csv | N=665


Computing MorganFP r2 n2048: 100%|██████████| 665/665 [00:00<00:00, 1291.56it/s]


[HalfLife_Human] Split 1/5 with seed 13
[HalfLife_Human] Split 2/5 with seed 17
[HalfLife_Human] Split 3/5 with seed 23
[HalfLife_Human] Split 4/5 with seed 29
[HalfLife_Human] Split 5/5 with seed 31
[HalfLife_Human] Metrics saved to results/HalfLife_Human_regression_metrics.csv
[HalfLife_Human] Parity and residual plots saved to figures/HalfLife_Human_*.png


In [ ]:
test_inference(smiles, "HalfLife_Human")

Computing MorganFP r2 n2048: 100%|██████████| 3/3 [00:00<00:00, 500.33it/s]



[0.507356   0.78613573 0.37344912]
[0.44893745 0.50217855 0.2366548 ]
[0.5558622  0.644727   0.40902734]
[0.5126523  0.5487356  0.44155174]
[0.726859   0.82352555 0.6436248 ]


In [ ]:
# D) hERG inhibition (binary)
herg_df = prepare_herg_classification()
run_classification_task("hERG_inhibition", herg_df, label_col="label")

print("\nAll tasks completed. Check the 'results/' CSVs and 'figures/' images.")

Downloading...


Loading TDC hERG...


100%|██████████| 885k/885k [00:00<00:00, 9.33MiB/s]
Loading...
Done!
Standardizing SMILES for hERG: 100%|██████████| 13445/13445 [00:43<00:00, 309.28it/s]


hERG processed saved to data/herg_processed.csv | N=13152; Positives=6574


Computing MorganFP r2 n2048: 100%|██████████| 13152/13152 [00:07<00:00, 1863.74it/s]


[hERG_inhibition] Split 1/5 with seed 13
[hERG_inhibition] Split 2/5 with seed 17
[hERG_inhibition] Split 3/5 with seed 23
[hERG_inhibition] Split 4/5 with seed 29
[hERG_inhibition] Split 5/5 with seed 31
[hERG_inhibition] Metrics saved to results/hERG_inhibition_classification_metrics.csv
[hERG_inhibition] ROC and PR curves saved to figures/hERG_inhibition_*.png

All tasks completed. Check the 'results/' CSVs and 'figures/' images.


In [ ]:
test_inference(smiles, "hERG_inhibition")

Computing MorganFP r2 n2048: 100%|██████████| 3/3 [00:00<00:00, 1413.49it/s]



[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
